# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.119743e+02     1.643724e+00
 * time: 0.09055089950561523
     1     1.034067e+01     9.426893e-01
 * time: 0.9903650283813477
     2    -1.138855e+01     9.973316e-01
 * time: 1.1008219718933105
     3    -3.398986e+01     7.560922e-01
 * time: 1.2773489952087402
     4    -4.730311e+01     6.407976e-01
 * time: 1.4274399280548096
     5    -5.677390e+01     2.893562e-01
 * time: 1.5643548965454102
     6    -5.961500e+01     2.541554e-01
 * time: 1.6771819591522217
     7    -6.085088e+01     1.086560e-01
 * time: 1.7840769290924072
     8    -6.142716e+01     4.188863e-02
 * time: 1.8977720737457275
     9    -6.172280e+01     3.488519e-02
 * time: 2.004142999649048
    10    -6.188543e+01     3.430444e-02
 * time: 2.118468999862671
    11    -6.202013e+01     2.046051e-02
 * time: 2.224565029144287
    12    -6.209378e+01     1.426287e-02
 * time: 2.338447093963623
    13    -6.214508e+01     1.149558e-02
 * time: 2.4449861049652

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671050
    AtomicLocal         -18.8557650
    AtomicNonlocal      14.8522647
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485352 
    Xc                  -19.3336812

    total               -62.261666461035
